In [1]:
import pandas as pd
import pretty_midi
import numpy as np
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder

In [51]:
meta=pd.read_csv('commu_meta.csv')

In [59]:
meta.head(10)

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,21,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",3,0.307692,0.708333,1,0,4,0,0,0.792,0.800,train,commu00001
1,0,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",1,0.307692,0.375000,0,2,0,0,0,0.168,0.224,train,commu00002
2,21,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",3,0.307692,0.958333,1,5,4,0,0,0.968,1.000,train,commu00003
3,0,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",3,0.307692,0.625000,1,4,7,0,0,0.344,0.352,train,commu00004
4,21,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",1,0.000000,0.208333,1,4,0,0,0,0.152,0.160,train,commu00005
5,21,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",3,0.307692,0.708333,1,5,4,0,0,0.952,0.960,train,commu00006
6,21,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",1,0.307692,0.625000,1,2,4,0,0,0.680,0.752,train,commu00007
7,0,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",3,0.000000,0.708333,1,1,5,0,0,0.528,0.720,train,commu00008
8,0,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",3,0.307692,0.708333,1,5,4,0,0,0.920,0.992,train,commu00009
9,21,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",1,0.307692,0.958333,1,2,4,0,0,0.888,0.960,train,commu00010


In [53]:
MAJOR_KEY = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
MINOR_KEY = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

KEY_MAP = {
    "cmajor": 0,
    "c#major": 1,
    "dbmajor": 1,
    "dmajor": 2,
    "d#major": 3,
    "ebmajor": 3,
    "emajor": 4,
    "fmajor": 5,
    "f#major": 6,
    "gbmajor": 6,
    "gmajor": 7,
    "g#major": 8,
    "abmajor": 8,
    "amajor": 9,
    "a#major": 10,
    "bbmajor": 10,
    "bmajor": 11,
    "cminor": 12,
    "c#minor": 13,
    "dbminor": 13,
    "dminor": 14,
    "d#minor": 15,
    "ebminor": 15,
    "eminor": 16,
    "fminor": 17,
    "f#minor": 18,
    "gbminor": 18,
    "gminor": 19,
    "g#minor": 20,
    "abminor": 20,
    "aminor": 21,
    "a#minor": 22,
    "bbminor": 22,
    "bminor": 23,
}

TIME_SIG_MAP = {
    "4/4": 0,
    "3/4": 1,
    "6/8": 2,
    "12/8": 3,
}
PITCH_RANGE_MAP = {
    "very_low": 0,
    "low": 1,
    "mid_low": 2,
    "mid": 3,
    "mid_high": 4,
    "high": 5,
    "very_high": 6,
}
INST_MAP = {
    "accordion": 1,
    "acoustic_bass": 3,
    "acoustic_guitar": 3,
    "acoustic_piano": 0,
    "banjo": 3,
    "bassoon": 5,
    "bell": 2,
    "brass_ensemble": 5,
    "celesta": 2,
    "choir": 7,
    "clarinet": 5,
    "drums_full": 6,
    "drums_tops": 6,
    "electric_bass": 3,
    "electric_guitar_clean": 3,
    "electric_guitar_distortion": 3,
    "electric_piano": 0,
    "fiddle": 4,
    "flute": 5,
    "glockenspiel": 2,
    "harp": 3,
    "harpsichord": 0,
    "horn": 5,
    "keyboard": 0,
    "mandolin": 3,
    "marimba": 2,
    "nylon_guitar": 3,
    "oboe": 5,
    "organ": 0,
    "oud": 3,
    "pad_synth": 4,
    "percussion": 6,
    "recorder": 5,
    "sitar": 3,
    "string_cello": 4,
    "string_double_bass": 4,
    "string_ensemble": 4,
    "string_viola": 4,
    "string_violin": 4,
    "synth_bass": 3,
    "synth_bass_808": 3,
    "synth_bass_wobble": 3,
    "synth_bell": 2,
    "synth_lead": 1,
    "synth_pad": 4,
    "synth_pluck": 7,
    "synth_voice": 7,
    "timpani": 6,
    "trombone": 5,
    "trumpet": 5,
    "tuba": 5,
    "ukulele": 3,
    "vibraphone": 2,
    "whistle": 7,
    "xylophone": 2,
    "zither": 3,
    "orgel": 2,
    "synth_brass": 5,
    "sax": 5,
    "bamboo_flute": 5,
    "yanggeum": 3,
    "vocal": 8,
}
GENRE_MAP = {
    "newage": 0,
    "cinematic": 1,
}
TRACK_ROLE_MAP = {
    "main_melody": 0,
    "sub_melody": 1,
    "accompaniment": 2,
    "bass": 3,
    "pad": 4,
    "riff": 5,
}
RHYTHM_MAP = {
    "standard": 0,
    "triplet": 1,
}


In [54]:
meta=meta.drop(meta.columns[[0]],axis=1)

In [55]:
scaler=MinMaxScaler()
meta[['num_measures','bpm','min_velocity','max_velocity']]=scaler.fit_transform(meta[['num_measures','bpm','min_velocity','max_velocity']])

In [56]:
def map_instruments(instrument_data):
    for key in INST_MAP:
        if key in instrument_data:
            return INST_MAP[key]  
meta['inst'] = meta['inst'].apply(map_instruments)

def map_time_sig(time_data):
    for key in TIME_SIG_MAP:
        if key in time_data:
            return TIME_SIG_MAP[key]
meta['time_signature'] = meta['time_signature'].apply(map_time_sig)

def map_track_role(track_role):
    for key in TRACK_ROLE_MAP:
        if key in track_role:
            return TRACK_ROLE_MAP[key]
meta['track_role'] = meta['track_role'].apply(map_track_role)

def map_genre(genre):
    for key in GENRE_MAP:
        if key in genre:
            return GENRE_MAP[key]
meta['genre'] = meta['genre'].apply(map_genre)

def map_pitch_range(pitch_range):
    for key in PITCH_RANGE_MAP:
        if key in pitch_range:
            return PITCH_RANGE_MAP[key]
meta['pitch_range'] = meta['pitch_range'].apply(map_pitch_range)

def map_rythm(rythm):
    for key in RHYTHM_MAP:
        if key in rythm:
            return RHYTHM_MAP[key]
meta['sample_rhythm'] = meta['sample_rhythm'].apply(map_rythm)

def map_audio_key(audio_key):
    for key in KEY_MAP:
        if key in audio_key:
            return KEY_MAP[key]
meta['audio_key'] = meta['audio_key'].apply(map_audio_key)